In [56]:
import urllib
import matplotlib
import networkit
from lxml import html
import re


def scrape_patent(url):      
    
    f = urllib.request.urlopen(url)
    raw_html = f.read()
    
    m = re.search('<TABLE>.*</TABLE>', str(raw_html))
    table_raw_html = m.group(0)
    
    doc = html.fromstring(table_raw_html)
    
    links = doc.xpath("//a/@href")
    links = list(set(links))
    print(len(links))
    
scrape_patent("http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=0&f=S&l=50&TERM1=Apple&FIELD1=AANM&co1=AND&TERM2=&FIELD2=&d=PTXT")

50
